## String Output Parser

In [2]:
from json import load
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)

template = PromptTemplate.from_template("Who is the President of countrry {country}")

chat = template.invoke({"country": "United States"})
result = llm.invoke(chat)
print(result.content)

As of my last knowledge update in October 2023, the President of the United States is Joe Biden. He was inaugurated on January 20, 2021. Please verify with up-to-date sources, as this information may change.


## Using Parser

In [3]:
parser = StrOutputParser()
template = PromptTemplate.from_template("Who is the President of countrry {country}")

chat = template.invoke({"country": "United States"})
result = llm.invoke(chat)
final_result = parser.invoke(result)
print(final_result)

As of my last update in October 2023, the President of the United States is Joe Biden. He took office on January 20, 2021. Please verify with up-to-date sources, as political positions can change.


## With chain

In [4]:
parser = StrOutputParser()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)
template = PromptTemplate.from_template("Who is the President of countrry {country}")

chain = template | llm | parser

result = chain.invoke({"country": "India"})

print(result)



As of my last update in October 2023, the President of India is Droupadi Murmu. She took office on July 25, 2022, becoming the first tribal woman to hold the position. Please verify with current sources for the most up-to-date information.


## Json Output Parser

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

json_parser = JsonOutputParser()

template = PromptTemplate.from_template("""
Extract the following product information from the text and retun it as a JSON:
- Name
-Price
- Category
-features (list)

text: {text}
format_instructions: {format_instructions}                                                                                                           
""",

partial_variables= {"format_instructions": json_parser.get_format_instructions() }
                                        
)

chain = template | llm | json_parser

text = """
The Samsung Galaxy S24 Ultra was launched at $1199. 
It belongs to the smartphone category. 
Main features include a 200MP camera, S-Pen support, Snapdragon 8 Gen 3 processor, and a 5000mAh battery.
"""
result = chain.invoke({"text": text})
print(result)

{'Name': 'Samsung Galaxy S24 Ultra', 'Price': 1199, 'Category': 'smartphone', 'features': ['200MP camera', 'S-Pen support', 'Snapdragon 8 Gen 3 processor', '5000mAh battery']}


## Pydantic Output Parser

In [7]:
from platform import release
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional

class Movie(BaseModel):
  titile:str = Field(..., description="The title of the movie")
  release_year: int = Field(..., description="Year the movie was released")
  genres: List[str] = Field(..., description="List of genres the movie belongs to")
  rating: float = Field(..., description="Average rating of the movie on a scale of 1 to 10")
  box_office: Optional[float] = Field(None, description="Worldwide box office in millions USD, if known")


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)

pydantic_parser = PydanticOutputParser(pydantic_object=Movie)

template = PromptTemplate.from_template("""Extract movie information from the folloing text: 
{text}
{format_instructions}""",
partial_variables={"format_instructions": pydantic_parser.get_format_instructions()})

chain = template | llm | pydantic_parser

text = """
The movie 'Inception', directed by Christopher Nolan, was released in 2010. 
It falls under the genres of science fiction, action, and thriller. 
It holds an average rating of 8.8 out of 10. 
The film grossed approximately $829.9 million at the global box office.
"""

result = chain.invoke({"text": text})
print(result)

titile='Inception' release_year=2010 genres=['science fiction', 'action', 'thriller'] rating=8.8 box_office=829.9
